In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import pathlib

In [2]:
sold_links =    {"Konika Big mini 302": "https://www.ebay.de/sch/i.html?_nkw=konika+big+mini+%28302%29&_sacat=0&LH_Complete=1&LH_Sold=1" , 
                 "Yashica T4": "https://www.ebay.de/sch/i.html?_nkw=yashica+%28T4%29&_ex_kw=zoomtec+90%2C+T3%2C+anleitung%2C+defekt%2C+ersatzteile%2C+Reparatur%2C+T2%2C+Nikon%2C+Chinon%2C+T5&_sacat=0&LH_Complete=1&LH_Sold=1" ,
                 "Contax T2": "https://www.ebay.de/sch/i.html?_nkw=contax+T2&_ex_kw=olympus%2C+canon%2C+nikon%2C+yashica%2C+konika%2C+defekt%2C+ersatzteile%2C+reparatur%2C+near+mint%2C++G%2C+distagon%2C+planar&_sacat=0&LH_Complete=1&LH_Sold=1" ,
                 "olympus MJU II all": "https://www.ebay.de/sch/i.html?_nkw=miju+II&_sacat=0&LH_Complete=1&LH_Sold=1"
                 }

active_links =  {"Konika Big mini 302": "https://www.ebay.de/sch/i.html?_from=R40&_trksid=p2334524.m570.l1313&_nkw=konika+big+mini+302&_sacat=0&LH_TitleDesc=0&_odkw=konika+big+mini+302+-mermaid&_osacat=0" ,
                 "Yashica T4": "https://www.ebay.de/sch/i.html?_nkw=yashica+%28T4%29&_ex_kw=zoomtec+90%2C+T3%2C+anleitung%2C+defekt%2C+ersatzteile%2C+Reparatur%2C+T2%2C+Nikon%2C+Chinon%2C+T5%2C+bastler%2C+prospekt%2C+manual&_sacat=0" ,
                 "Contax T2": "https://www.ebay.de/sch/i.html?_nkw=contax+T2&_ex_kw=olympus%2C+canon%2C+nikon%2C+yashica%2C+konika%2C+defekt%2C+ersatzteile%2C+reparatur%2C+near+mint%2C++G%2C+distagon%2C+planar&_sacat=0" ,
                 "Olympus MJU II all": "https://www.ebay.de/sch/i.html?_nkw=miju+II&_sacat=0"
                 }


### Next cell is for running the script on a single Url, to test 

In [5]:
driver = webdriver.Firefox()
driver.implicitly_wait(10)
url = "https://www.ebay.de/sch/i.html?_nkw=konika+big+mini+%28302%29&_sacat=0&LH_Complete=1&LH_Sold=1" 
driver.get(url)
# Click on the coockies preference banner after some time to be sure the element is clickable
time.sleep(2)
# close Ebay pop-up
try:
    driver.find_element(By.ID, "gdpr-banner-accept").click()
except:
    pass
# Get the page source and get the content of it with BeautifulSoup
page_source = driver.page_source
soup = BeautifulSoup(page_source)

# We want to retrieve the results of the search, which are stored in an 
#unordered-list (ul) html item and we specify a selector ("class") to choose the right elem.
results = soup.find("ul", {"class" : "srp-results srp-list clearfix"})
#results.text

In [7]:
#The text in previous output isn't helpful. 
# We want to process each item of the unordered list, more than having the whole output 
#in a single string:
items = results.findAll("li")
for item in items:
    #print(item)
    itemBreak = item.find("li", {"class" : "srp-river-answer"})
    print(itemBreak)
    itemTitle = item.find("div", {"class" : "s-item__title"})
    itemPrice = item.find("span", {"class": "s-item__price"})
    itemDate = item.find("span", {"class" : "POSITIVE"}) #POSITIVE=sold, NEGATIVE=ended (no bids)
    # There are some itemTitle None. Exclude them
    if itemTitle:
        # Delete the annoying "NEW LISTING" Tag:
        newListingTag = itemTitle.find("span", {"class" : "LIGHT_HIGHLIGHT"})
        if newListingTag:
            newListingTag.extract()
        itemTitle = itemTitle.text
        print(itemTitle)
    if itemPrice:
        itemPrice = itemPrice.text
        itemPrice = float(itemPrice.split()[-1].replace(".", "").replace(",","."))
        print(itemPrice)
    if itemDate:
        # Eliminate "verkauft" from the date
        itemDate = itemDate.text
        itemDate = itemDate.split()[1:]
        # HARDCODED: Change Dez to Dec
        itemDate[1] = "Dec" if itemDate[1] == "Dez" else itemDate[1]
        itemDate[1] = "Oct" if itemDate[1] == "Okt" else itemDate[1]
        itemDate[1] = "Mar" if itemDate[1] in ("Mrz", "März") else itemDate[1]
        # Apply date formatting %b %d, %Y
        itemDate = itemDate[1]+" "+itemDate[0][:-1]+", "+itemDate[2]
        # Create a datetime object to store the date
        itemDate = datetime.strptime(itemDate, "%b %d, %Y")
        print(itemDate.strftime("%B %d, %Y"))
        
    if "Ergebnisse für weniger Suchbegriffe" in item.strings:
        break

None
None
None
None
None
None
Konica Big Mini BM-302 Compact Film Camera - 35mm f/3.5
195.0
April 03, 2023
None
Kamera Konica Big Mini BM-302
98.0
April 01, 2023
None
Konica Big Mini BM-302 compact argentique 35mm Point & Shoot
162.0
March 28, 2023
None
KONICA BIG MINI BM 302 … ANALOGKAMERA - FOTOAPARAT
19.0
March 16, 2023
None
Konica Big Mini BM-302 Point & Shoot Camera - Very Good Condition
168.0
March 10, 2023
None
Kamera Konica Big Mini BM-302 
74.0
March 01, 2023
None
Compact Point & Shoot Konica Big Mini BM-302 35 mm 1:3,5 with case
160.0
February 24, 2023
None
Konica Big mini BM-302 Kompaktkamera
29.5
February 22, 2023
None
Konica Big Mini BM 302, als Ersatzteil , für Bastler, defekt, inkl Tasche
20.0
February 16, 2023
None
✮ Konica Big Mini BM-302 mit 35mm f/3.5 Objektiv #2662635 || vom Händler!
169.0
February 07, 2023
None
Kamera Konica Big Mini BM-302
159.0
February 01, 2023
None
Kamera Konica Big Mini BM-302 auto date 35mm f3,5 Vintage 90er analog Foto 
140.0
January 31, 202

In [8]:
# Same as previous cell, but with integration of pandas in mind:
def convertResultsToPandas(page_source, model: str = None):
    soup = BeautifulSoup(page_source)
    results = soup.find("ul", {"class" : "srp-results srp-list clearfix"})
    # Create empty list (or numpy array) for storing the rows of df
    rows = []
    items = results.findAll("li")
    for item in items:
        if "Ergebnisse für weniger Suchbegriffe" in item.strings:
            break
        itemTitle = item.find("div", {"class" : "s-item__title"})
        itemPrice = item.find("span", {"class": "s-item__price"})
        itemDate = item.find("span", {"class" : "POSITIVE"}) #POSITIVE=sold, NEGATIVE=ended (no bids)
        # There are some itemTitle None. Exclude them
        if itemTitle and itemPrice and itemDate:
            # Delete the annoying "NEW LISTING" Tag:
            newListingTag = itemTitle.find("span", {"class" : "LIGHT_HIGHLIGHT"})
            if newListingTag:
                newListingTag.extract()
            itemTitle = itemTitle.text
            #print(itemTitle)
            
            # Transform the price string to float
            itemPrice = itemPrice.text
            itemPrice = float(itemPrice.split()[-1].replace(".", "").replace(",","."))
            #print(itemPrice)
    
            # Eliminate "verkauft" from the date
            itemDate = itemDate.text
            itemDate = itemDate.split()[1:]
            # HARDCODED: Change Dez to Dec
            itemDate[1] = "Dec" if itemDate[1] == "Dez" else itemDate[1]
            itemDate[1] = "Oct" if itemDate[1] == "Okt" else itemDate[1]
            itemDate[1] = "Mar" if itemDate[1] in ("Mrz", "März") else itemDate[1]
            # Apply date formatting %b %d, %Y
            itemDate = itemDate[1]+" "+itemDate[0][:-1]+", "+itemDate[2]
            # Create a datetime object to store the date
            itemDate = datetime.strptime(itemDate, "%b %d, %Y")
            #print(itemDate.strftime("%B %d, %Y"))
            
            # Now create a row for the df from the extracted data
            row = [itemTitle, itemPrice, itemDate]
            rows.append(row)
    df = pd.DataFrame.from_records(rows, columns=["Title", "Price [€]", "Purchase Date"])
    df.assign(Model = model)
    return df   


In [5]:
# If we automate webdriver operation in the function we can omitt the condition:

def convertEbaySearchtoPandas(url):
    # Webdriver operation: open Browser, go to Url, accept Ebay Banner
    driver = webdriver.Firefox()
    driver.implicitly_wait(10)
    driver.get(url)
    time.sleep(3)
    # close Ebay pop-up
    try:
        driver.find_element(By.ID, "gdpr-banner-accept").click()
    except:
        pass
    
    # Initialize empty Data Frame
    df_total = pd.DataFrame(columns=["Title", "Price [€]", "Purchase Date"])

    while(True):
        df = convertResultsToPandas(driver.page_source)
        #print(df)
        df_total = pd.concat([df_total, df])
        
        # HARDCODED: set isDisabledNextPage to True for the case an Exception is raised
        isDisabledNextPage = True
        try:
            # Find the "NextPage" button
            nextPageLink = driver.find_element(By.CLASS_NAME, "pagination__next") #Xpath: "//a[@class='pagination__next icon-link']"
            # Get the attribute abled/disabled
            isDisabledNextPage = nextPageLink.get_attribute("aria-disabled") == "true"
        except NoSuchElementException:
            print("Results consist of only 1 page. Exiting...\n")
        
        if isDisabledNextPage:
            print("Reached last page. Exiting...\n")
            break
        else:
            nextPageLink.click()
            time.sleep(2)
    
    driver.close()
    
    return df_total

In [ ]:
def save_to_disk():
    pass

In [16]:
url = "https://www.ebay.de/sch/i.html?_dcat=27386&_fsrp=1&_from=R40&_nkw=rtx+3070&_sacat=0&LH_TitleDesc=0&Speichergr%25C3%25B6%25C3%259Fe=8%2520GB&rt=nc&LH_Sold=1&LH_Complete=1" 
df_rtx_3060 = convertEbaySearchtoPandas(url, "RTX 3070")
url = "https://www.ebay.de/sch/i.html?_from=R40&_nkw=blackmagic+pocket+cinema+camera+4k&_sacat=0&LH_TitleDesc=0&_fsrp=1&LH_Complete=1&LH_Sold=1&Marke=Blackmagic%7CBlackmagic%2520Design&_dcat=11724&rt=nc&LH_PrefLoc=3"
df_bmpcc_4k = convertEbaySearchtoPandas(url, "BMPCC 4K")
url = "https://www.ebay.de/sch/i.html?_from=R40&_nkw=nikon+F2&_sacat=0&LH_Sold=1&LH_Complete=1&rt=nc&Modell=Nikon%2520F2&_dcat=15230"
df_nikon_F2 = convertEbaySearchtoPandas(url, "Nikon F2")

In [19]:
df_nikon_F2.to_csv("nikon_F2.csv")
df_bmpcc_4k.to_csv("bmpcc_4k.csv")
df_rtx_3060.to_csv("RTX_3070.csv")